<a href="https://colab.research.google.com/github/aalimkeskinn/basicjs-turkce/blob/main/demo/vibevoice_realtime_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VibeVoice-Realtime Colab — T4 Quickstart



## Step 1: Setup Environment

In [ ]:
# Check for T4 GPU
import torch
if torch.cuda.is_available() and "T4" in torch.cuda.get_device_name(0):
    print("✅ T4 GPU detected")
else:
    print("""
    ⚠️ WARNING: T4 GPU not detected

    The recommended runtime for this Colab notebook is "T4 GPU".

    To change the runtime type:

        1. Click on "Runtime" in the top navigation menu
        2. Click on "Change runtime type"
        3. Select "T4 GPU"
        4. Click "OK" if a "Disconnect and delete runtime" window appears
        5. Click on "Save"

    """)

# Clone the VibeVoice repository
![ -d /content/VibeVoice ] || git clone --quiet --branch main --depth 1 https://github.com/microsoft/VibeVoice.git /content/VibeVoice
print("✅ Cloned VibeVoice repository")

# Install project dependencies
!uv pip --quiet install --system -e /content/VibeVoice
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared && chmod +x cloudflared
print("✅ Installed dependencies")

# Download model
from huggingface_hub import snapshot_download
snapshot_download("microsoft/VibeVoice-Realtime-0.5B", local_dir="/content/models/VibeVoice-Realtime-0.5B")
print("✅ Downloaded model: microsoft/VibeVoice-Realtime-0.5B")


[Optional] If the download exceeds 1 minute, it is probably stuck. You can: (1) interrupt the execution, (2) log in to Hugging Face, and (3) try download again.

In [ ]:
from huggingface_hub import login
login()

In [ ]:
snapshot_download("microsoft/VibeVoice-Realtime-0.5B", local_dir="/content/models/VibeVoice-Realtime-0.5B")
print("✅ Downloaded model: microsoft/VibeVoice-Realtime-0.5B")

[Optional] More experimental voices

In [ ]:
!bash /content/VibeVoice/demo/download_experimental_voices.sh

## Step 2: Launch VibeVoice-Realtime Demo

In [ ]:
import subprocess, re, time, threading

srv = subprocess.Popen(
    "python /content/VibeVoice/demo/vibevoice_realtime_demo.py --model_path /content/models/VibeVoice-Realtime-0.5B --port 8000",
    shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True,
)
cf = subprocess.Popen(
    "./cloudflared tunnel --url http://localhost:8000 --no-autoupdate",
    shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True,
)

public_url = None
server_ready = False
url_pattern  = re.compile(r"(https://[a-z0-9-]+\.trycloudflare\.com)")

def read_srv():
    global server_ready
    for ln in srv.stdout:
        print(ln.strip())
        if "Uvicorn running on" in ln:
            server_ready = True

def read_cf():
    global public_url
    for ln in cf.stdout:
        m = url_pattern.search(ln)
        if m:
            public_url = m.group(1)
            break

threading.Thread(target=read_srv, daemon=True).start()
threading.Thread(target=read_cf,  daemon=True).start()


while True:
    if server_ready and public_url:
        print(f"✅ Public URL: {public_url}\n");
        public_url = None
    time.sleep(0.25)

Görüntülenen çıkış son 5000 satıra kısaltıldı.
Prefilled 1393 text tokens, generated 3206 speech tokens, current step (4857 / 8192):  59%|█████▉    | 4857/8192 [06:22<06:04,  9.15it/s]
Prefilled 1393 text tokens, generated 3206 speech tokens, current step (4857 / 8192):  59%|█████▉    | 4858/8192 [06:22<06:15,  8.89it/s]
Prefilled 1393 text tokens, generated 3207 speech tokens, current step (4858 / 8192):  59%|█████▉    | 4858/8192 [06:22<06:15,  8.89it/s]
Prefilled 1393 text tokens, generated 3207 speech tokens, current step (4858 / 8192):  59%|█████▉    | 4859/8192 [06:22<06:12,  8.95it/s]
Prefilled 1393 text tokens, generated 3208 speech tokens, current step (4859 / 8192):  59%|█████▉    | 4859/8192 [06:22<06:12,  8.95it/s]
Prefilled 1393 text tokens, generated 3208 speech tokens, current step (4859 / 8192):  59%|█████▉    | 4860/8192 [06:22<06:06,  9.09it/s]
Prefilled 1393 text tokens, generated 3209 speech tokens, current step (4860 / 8192):  59%|█████▉    | 4860/8192 [06:22<06:06